In [9]:
%matplotlib inline
from numpy import *
from matplotlib.pyplot import *

In [10]:
n = 30
h = 1./(n-1)

x = linspace(0,1,n)

a = -ones((n-1,))
b = 2*ones((n))

A = (1/h**2)*(diag(a,-1) + diag(b) + diag(a,+1))
f = x*(1-x)

A[0,:] = 0
A[:,0] = 0
A[0,0] = 1
f[0] = 0

A[-1,:] = 0
A[:,-1] = 0
A[-1,-1] = 1
f[-1] = 0

# exact solution
u = linalg.solve(A, f)

In [11]:
# Jacobi method
def jacobi(A, b, nmax=10000, eps=1e-10):
  N = len(A)
  x = zeros_like(b)
  x_old = zeros_like(b)
  tol = eps+1
  it = 0
  while(it<nmax and tol>eps):
    it += 1
    for i in range(N):
      x[i] = 1./A[i,i]*(b[i]-dot(A[i,0:i], x_old[0:i]) - dot(A[i,i+1:N],x_old[i+1:N]))

    res = b - dot(A,x)
    tol = linalg.norm(res,2)

    x_old = x.copy()

  print(it, tol)
  return x

In [12]:
sol_jacobi = jacobi(A, f)
print(linalg.norm(sol_jacobi-u)/linalg.norm(u))

3914 9.965885316539395e-11
1.01435369692299e-10


In [14]:
# Gauss Seidel method
def gauss_seidel(A, b, nmax=10000, eps=1e-10):
  N = len(A)
  x = zeros_like(b)
  x_old = zeros_like(b)
  tol = eps+1
  it = 0
  while(it<nmax and tol>eps):
    it += 1
    for i in range(N):
      x[i] = 1./A[i,i]*(b[i]-dot(A[i,0:i], x[0:i]) - dot(A[i,i+1:N],x_old[i+1:N]))

    res = b - dot(A,x)
    tol = linalg.norm(res,2)

    x_old = x.copy()

  print(it, tol)
  return x

In [15]:
sol_gauss_seidel = gauss_seidel(A, f)
print(linalg.norm(sol_gauss_seidel-u)/linalg.norm(u))

1958 9.975573046289024e-11
1.0068983192453529e-10


In [16]:
# Preconditioned gradient method

def grandient(A, b, P, nmax=8000, eps=1e-10):
  n = len(A)
  x = zeros_like(b)
  tol = eps+1
  r = b - dot(A,x)
  count = 0
  while(count<nmax and tol > eps):
    z = linalg.solve(P,r)
    alpha = dot(r,z) / (dot(z,dot(A,z)))
    x = x + alpha*z
    r = r - alpha*dot(A,z)
    tol = linalg.norm(r,2)
    count = count + 1
  print(count, tol)
  return x

In [18]:
sol_grad = grandient(A, f, identity(len(A)))
print(linalg.norm(sol_grad - u)/linalg.norm(u))

sol_pre_grad = grandient(A, f, A)
print(linalg.norm(sol_pre_grad - u)/linalg.norm(u))

3183 9.950273670244494e-11
7.477371613409447e-11
1 1.2400191698686355e-14
1.9043722965187744e-15


In [19]:
# Precoditioned conjugate gradient method
def conj_grad(A, b, P, nmax=len(A), eps=1e-10):
  N = len(A)
  x = zeros_like(b)
  tol = eps+1
  it = 0
  r = b - dot(A,x)
  p_old = zeros_like(b)
  rho_old = 1.
  while (it<nmax and tol>eps):
    it += 1
    z = linalg.solve(P,r)
    rho = dot(r,z)
    if (it>1):
      beta = rho/rho_old
      p = z + beta*p_old
    else:
      p = z
    q = dot(A,p)
    alpha = rho/dot(p,q)
    x += p*alpha
    r -= q*alpha

    p_old = p
    rho_old = rho

    tol = linalg.norm(r,2)

  print(it, tol)
  return x

In [21]:
sol_conj_grad = conj_grad(A, f, identity(len(A)))
print(linalg.norm(sol_conj_grad-u)/linalg.norm(u))

sol_pre_conj_grad = conj_grad(A, f, A)
print(linalg.norm(sol_pre_conj_grad-u)/linalg.norm(u))

14 5.944685519295643e-16
1.1809539624857359e-15
1 1.2400191698686355e-14
1.9043722965187744e-15
